In [1]:
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.local import LocalQuantumJob
from braket.jobs.image_uris import Framework, retrieve_image
import time
import pickle

# aws_session = AwsSession(default_bucket="amazon-braket-eu-west-2-lukasvoss")
aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

In [3]:
# "arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy"

device = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"

parts = device.split('/')
# Access the last element
device_name = parts[-1]

In [4]:
num_total_updates = 50

hyperparams = {
    "num_total_updates": num_total_updates, 
}

#Name your job so that it can be later identified
job_name = f'cx-gate-calibration-{num_total_updates}-updates-{str(int(time.time()))}-{device_name}'

# Specify one of the ready-to-use containers provided by Amazon Braket
# image_uri = retrieve_image(Framework.PL_PYTORCH, AwsSession().region)

q_env_config_path = 'config_yamls/q_env_gate_config.yaml'
agent_config_path = 'config_yamls/agent_config.yaml'

input_config = {
    'q-env-config': q_env_config_path,
    'agent-config': agent_config_path,
}

In [5]:
job = AwsQuantumJob.create(
    #Run this hybrid job on the SV1 simulator
    device=device,
    #The directory or single file containing the code to run.
    source_module="needed_files",
    #The main script or function the job will run.
    entry_point="needed_files.algorithm_script:calibrate_gate",
    #Set the name of the job
    job_name=job_name,
    # Set the hyperparameters
    hyperparameters=hyperparams,
    # Specify the image to use for the container to run the script in
    # image_uri=image_uri,
    # Define the file that contains the input data but not relevant for the minimal example
    input_data=input_config,
    wait_until_complete=True,
    aws_session=aws_session,
)

Initializing Braket Job: arn:aws:braket:us-west-1:229406906664:job/cx-gate-calibration-10-updates-1706887825-sv1
........................
2024-02-02 15:32:57,144 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-02-02 15:32:57,144 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-02-02 15:32:57,157 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-02-02 15:32:57,157 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-02-02 15:32:57,168 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-02-02 15:32:57,169 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-02-02 15:32:57,180 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "agent-config": "/opt/ml/i

In [6]:
job.result()

{}

In [7]:
while job.state() not in job.TERMINAL_STATES:
    print(job.state())
    time.sleep(30)

print(job.state())
print(job.result())

CANCELLED
{}


In [8]:
pickle_name = job_name + '.pickle'

with open(job_name, 'wb') as handle:
    pickle.dump(job.result(), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
avg_reward = job.result()['avg_reward']
std_actions = job.result()['std_actions']
fidelities = job.result()['fidelities']

KeyError: 'avg_reward'

#### Visualize results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [ ]:
# plt.plot(np.arange(1, len(avg_reward)+1), avg_reward, label='mean reward')
# plt.plot(np.arange(2, len(fidelities)+2), fidelities, label='gate fidelity')
# plt.xlabel('update step')
# plt.legend()

In [ ]:
plt.plot(np.arange(1, num_total_updates+1), avg_reward, label='mean reward')
plt.plot(np.arange(2, num_total_updates+1), fidelities, label='gate fidelity')
plt.xlabel('update step')
plt.legend()

In [ ]:
print(f'Final Gate Fidelity: {fidelities[-1]:.4%}')
print(f'\nMax Gate Fidelity: {max(fidelities):.4%}')

In [ ]:
std_actions_componentwise = list(zip(*std_actions))

In [ ]:
for ind, param in enumerate(std_actions_componentwise):
    plt.plot(np.arange(1, num_total_updates+1), param, label=r'std $\theta_{}$'.format(ind+1))

plt.xlabel('update step')
plt.legend()

#### Best Action Vector

In [ ]:
best_action_vector = job.result()['action_vector']

In [ ]:
best_action_vector